# Simpy - System Simulation in Python

`$ pip install simpy faker`

# Números aleatorios

- [Relationship between Poisson and Exponential distribution](https://stats.stackexchange.com/questions/2092/relationship-between-poisson-and-exponential-distribution)
- [random.expovariate](https://docs.python.org/3/library/random.html#random.expovariate)

Podemos probar que se cumple la media generando 10K números y computandola.

In [1]:
import random
mean = 5.4
numeros = [random.expovariate(1.0 / mean) for i in range(0, 10000)]
numeros[:3]

[2.114481934028434, 4.436433497741162, 5.85384286054457]

In [2]:
from pandas import Series
series = Series(numeros)
print("Media: %f" % series.mean())

Media: 5.391090


# Strings Aleatorios

In [3]:
from faker import Factory
fake = Factory.create('es-ES')
print(fake.name())
print(fake.address())

Noelia Ramírez Sola
Callejón de Manuel Laguna 3 Puerta 3 
Huelva, 34822


# Simulación cola básica M/M/1

Definiremos dos listas para guardar los tiempos de espera y de servicio

In [4]:
t_espera = []
t_servicio = []

## Estación

Podemos entonces definir una funciona para simular el tiempo de servicio de un cliente

In [5]:
def cliente_en_servicio(env, nombre, tiempo_sistema):
    inicio_servicio = env.now
    tiempo_servicio = random.expovariate(1.0 / tiempo_sistema)
    yield env.timeout(tiempo_servicio)
    t_servicio.append(tiempo_servicio)
    print('%7.4f %s servido en %f' % (env.now, nombre, tiempo_servicio))

Importando un entorno de Simpy, podemos llamar a la función para obtener el tiempo de servicio

In [6]:
import simpy
env = simpy.Environment()

list(cliente_en_servicio(env, fake.first_name(), 1.0))

 0.0000 Ismael servido en 0.264852


[<Timeout(0.2648520695050856) object at 0x1106dd860>]

## Cola y estación

A partir de aqui, podemos simular un cliente completo que espera en cola, entra en la estación y abandona el sistema:

In [7]:
def cliente(env, nombre, cola, tiempo_sistema):
    llegada = env.now
    print('%7.4f %s se pone en cola' % (llegada, nombre))

    with cola.request() as req:        
        yield req
        
        espera = env.now - llegada
        t_espera.append(espera)    
        print('%7.4f %s en sistema, espera: %f' % (env.now, nombre, espera))
        
        yield cliente_en_servicio(env, nombre, tiempo_sistema)

In [8]:
env = simpy.Environment()
cola = simpy.Resource(env, capacity=1)

list(cliente(env, fake.first_name(), cola, 1.0))

 0.0000 Joan se pone en cola
 0.0000 Joan en sistema, espera: 0.000000


[<Request() object at 0x1107102b0>,
 <generator object cliente_en_servicio at 0x1106f0fc0>]

## Orígen de clientes

In [9]:
def source(env, number, intervalo_llegadas, cola, tiempo_sistema):
    for i in range(number):
        c = cliente(env, fake.first_name(), cola, tiempo_sistema)
        env.process(c)
        t = random.expovariate(1.0 / intervalo_llegadas)
        yield env.timeout(t)

In [10]:
TOTAL_CLIENTES = 10
TIEMPO_SISTEMA = 5
INTERVALO_LLEGADAS = 3

## Simulación Completa

In [11]:
random.seed(42)
env = simpy.Environment()
cola = simpy.Resource(env, capacity=1)
env.process(source(env, TOTAL_CLIENTES, INTERVALO_LLEGADAS, cola, TIEMPO_SISTEMA))
# env.run()

<Process(source) object at 0x110710fd0>